# Simple ATLAS Analysis

Crude form to fetch ATLAS data and plot it

In [12]:
from func_adl_servicex import ServiceXSourceUpROOT
from func_adl import ObjectStream
from hist import Hist
import mplhep as mpl
import awkward as ak

from utils import files

In [6]:
def make_ds(name: str):
    '''Create a ServiceX Datasource for a particular ATLAS Open data file
    '''
    return ServiceXSourceUpROOT(files[name]['files'], files[name]['treename'], backend='open_uproot')

In [13]:
def apply_event_cuts (source: ObjectStream) -> ObjectStream:
    '''Event level cuts for the analysis. Keep from sending data that we aren't going to need at all in the end.
    '''
    return (source
        .Where(lambda e: e.trigE or e.trigM))

In [ ]:
def good_leptons(source: ObjectStream) -> ObjectStream:
    '''Select out all good leptons from each event. Return their pt, eta, phi, and E, and other
    things needed downstream.

    Because uproot doesn't tie toegher the objects, we can't do any cuts at this point.
    '''
    return source.Select(lambda e: {
        'lep_pt': e.lep_pt,
        'lep_eta': e.lep_eta,
        'lep_phi': e.lep_phi,
        'lep_E': e.lep_E,
        'lep_ptcone30': e.lep_ptcone30,
        'lep_etcone20': e.lep_etcone20,
        'lep_type': e.lep_type,
        'lep_trackd0pvunbiased': e.lep_trackd0pvunbiased,
        'lep_tracksigd0pvunbiased': e.lep_tracksigd0pvunbiased,
        'lep_z0': e.lep_z0,
    })    

In [16]:
ds = make_ds('ggH125_ZZ4lep')
good_events = apply_event_cuts(ds)

In [19]:
data = (good_events
    .Select("lambda e: {'lep_pt': e['lep_pt']}")
    .AsAwkwardArray()
    .value())

root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2020-01-22/4lep/MC/mc_345060.ggH125_ZZ4lep.4lep.root:   0%|          | 0/9000000000.0 [00:00]
root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2020-01-22/4lep/MC/mc_345060.ggH125_ZZ4lep.4lep.root:   0%|          | 0/1 [00:00]
        Downloaded:   0%|          | 0/1 [00:00]           
root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2020-01-22/4lep/MC/mc_345060.ggH125_ZZ4lep.4lep.root:   0%|          | 0/1 [00:05]
        Downloaded:   0%|          | 0/1 [00:05]
root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2020-01-22/4lep/MC/mc_345060.ggH125_ZZ4lep.4lep.root:   0%|          | 0/1 [00:10]
        Downloaded:   0%|          | 0/1 [00:10]
                                                                                                                                                
        Downloaded: 100%|██████████| 1/1 [00:17]
                                                

In [20]:
len(data['lep_pt']), len(ak.flatten(data['lep_pt']))

(164715, 659802)

Some info on how the cuts are going

| cuts | # Events | # Leptons |
| -: | -: | -: |
| None | 164,716 | 659806 |
| E/M Trigger |  164,715 | 659802 |


In [23]:
ak.count(data['lep_pt'], axis=1)

<Array [4, 4, 4, 4, 4, 4, ... 4, 4, 4, 4, 4, 4] type='164715 * int64'>